## Add source code folder to PATH

In [3]:
import os
import sys

sys.path.append(os.path.abspath("../labs"))

## Django setup

In [4]:
from django.core.wsgi import get_wsgi_application

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "True")

os.environ.setdefault("LOCAL_LLM_HOST", "http://localhost:11434")
os.environ.setdefault("DATABASE_HOST", "localhost")

application = get_wsgi_application()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/runtime/Documents/Projectos/labs/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/runtime/Documents/Projectos/labs/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/runtime/Documents/Projectos/labs/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /BerriAI/litellm/main/model_prices_and_context_window.json HTTP/1.1" 200 12946
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_location

In [5]:
ISSUE = "ADD YOUR ISSUE TEXT HERE"
EMBEDDER_MODEL_NAME = "nomic-embed-text:latest"
LLM_MODEL_NAME = "llama3.2:latest"
REPO = "."

## Embeddings

In [4]:
from embeddings.embedder import Embedder
from embeddings.ollama import OllamaEmbedder

embeedder = Embedder(OllamaEmbedder, EMBEDDER_MODEL_NAME)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/runtime/Documents/Projectos/labs/.venv/lib/python3.12/site-packages/certifi/cacert.pem'


## Vectorize

In [5]:
from embeddings.vectorizers.base import Vectorizer
from embeddings.vectorizers.chunk_vectorizer import ChunkVectorizer

Vectorizer(ChunkVectorizer, embeedder).vectorize_to_database(None, repo_destination=REPO)

DEBUG:embeddings.vectorizers.chunk_vectorizer:Loading and splitting all documents into chunks.
DEBUG:embeddings.vectorizers.chunk_vectorizer:Embedding all repo documents.
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x120efdf70>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.failed exception=KeyboardInterrupt()
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete


KeyboardInterrupt: 

## Embeddings:lookup

In [ ]:
embeddings_results = embeedder.retrieve_embeddings(ISSUE, REPO)

similar_embeddings = [(embedding.repository, embedding.file_path, embedding.text) for embedding in embeddings_results]

In [ ]:
similar_embeddings

## Prompt

In [ ]:
from llm import get_prompt, prepare_context

prompt = get_prompt(ISSUE)
prepared_context = prepare_context(similar_embeddings, prompt)

## LLM

In [ ]:
from litellm_service.llm_requester import Requester
from litellm_service.ollama import OllamaRequester
from llm import validate_llm_response

requester = Requester(OllamaRequester, model=LLM_MODEL_NAME)
llm_response = requester.completion_without_proxy(prepared_context)
redo, redo_reason = validate_llm_response(llm_response)

In [ ]:
from pprint import pprint

pprint(llm_response[1]["choices"][0]["message"]["content"].replace("\\n", " "))